## Free Time-dependent Schrödinger equation

We consider the time-dependent Schrödinger equation for a free particle:

$$
i \partial_t u(x,y,t) = - \Delta u(x,y,t)
$$

with homogeneous Dirichlet boundary conditions:

$$
u(x,y,t) = 0 \quad \text{on } \partial \Omega, t \geq 0
$$

and initial condition:

$$
u(x,y,0) = u_0(x,y)
$$

where

$$
u_0(x,y) = \sin(\pi x) \sin(\pi y)
$$

The exact solution is:

$$
u_{exact}(x,y,t) = \sin(\pi x) \sin(\pi y) e^{-2 i \pi^2 t}
$$

Proof:

In the following, we will assume $\Omega = [0,1]^2$. We take the ansatz:

$$
u(x,y,t) = \phi(x,y) e^{-i \lambda t}
$$

Substituting this into the Schrödinger equation, we get:

$$
i \partial_t \phi(x,y) e^{-i \lambda t} = - \Delta \phi(x,y) e^{-i \lambda t}
$$

which results in:

$$
\lambda \phi(x,y) = - \Delta \phi(x,y)
$$

The eigenfunctions of the Dirichlet Laplacian on the unit square are the sine functions:

$$
\phi_{n,m}(x,y) = \sin(n \pi x) \sin(m \pi y)
$$

where $n,m \in \mathbb{N}$ and $\lambda_{n,m} = \pi^2 (n^2 + m^2)$. From the initial condition, we know that $n = m = 1$.

Therefore, the exact solution is:

$$
u_{exact}(x,y,t) = \sin(\pi x) \sin(\pi y) e^{-2 i \pi^2 t}
$$

## Method of manufactured solutions



